<!DOCTYPE html>
<html>
    <head>
        <meta name=\"viewport\" content=\"width=device-width initial-scale=1\">
        <style>
            * {
            box-sizing: border-box;
            }
            .row {
            display: flex;
            }
            .column {
            flex: 33.33%;
            padding: 5px;
            }
        </style>
    </head>
    <body>
        <div class="row">
            <div class="column">
                <img src="http://www.lncc.br/images/logo_lncc.png" alt="LNCC" style="width:18%" align="left">
            </div>
            <div class="column">
                <img src="http://dexl.lncc.br/files/logo_dexl_rgb.jpg" alt="Dexl" style="width:22%;margin-left:400px;margin-top:10px">
            </div>
            <div class="column">
                <img src="https://upload.wikimedia.org/wikipedia/pt/7/71/Cnpq-logo.jpg" alt="CENPES" style="width:20%;margin-top:-47px" align="right">
            </div>
        </div>
        <hr>
        <div align="middle">
            <h2>Desenvolvimento de Web Apps via Shiny R</h2>
            <h3>EAMC 2020</h3>
            <h4>Instrutor: Rafael Silva Pereira</h4> 
        </div>
    </body>
</html>

Neste notebook iremos discutir uma segunda forma de trabalhar com processamento paralelo de dados, utilizando o processo map reduce

Apresentamos aqui duas funções em especial

map: aplica uma função sobre o dado particionado e retorna uma lista aonde cada indice é o calculo em uma partição

Reduce: Recebe como input uma lista e aplica uma função recursiva sobre esta, se torna util para agregar os dados processados pela função map, map é uma função incluida no pacote purrr, membro do tidyverse

In [1]:
require(tidyverse)

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  2.0.1     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.4.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
map(c(9,16,25),sqrt)

[[1]]
[1] 3

[[2]]
[1] 4

[[3]]
[1] 5

Como exemplo carreguemos o script do filme aliens para lermos

In [4]:
source("~/scripts_R/Analise_texto.R")

In [5]:
Filme=leitura("../Aliens.pdf")

Loading required package: NLP

Attaching package: ‘NLP’

The following object is masked from ‘package:ggplot2’:

    annotate



In [6]:
Filme

[1] "    Bio-readouts are all in the green. Looks like she’s alive. Well, there goes our\nsalvage, guys. How are we today? Terrible. Well, better than yesterday, at least.\n- Where am I? - You’re safe. You’re at Gateway Station. Been here a couple of\ndays. You were pretty groggy at first, but now you’re OK. Looks like you’ve got\na visitor. Jonesy! Come here! Hey, come here! How are you, you stupid cat?\nHow are you! Where have you been? Guess you two have met, huh? I’m Burke.\nCarter Burke. I work for the company. But... Don’t let that fool you. I’m really\nan OK guy. I’m glad to see you’re feeling better. They say that all the weakness\nand disorientation should pass soon. That’s just natural side effects of such a\nlong hypersleep. Or something like that. What do you mean? How long was I\nout there? Has no-one discussed this with you yet? No. But, I mean, I don’t...\n... Recognise this place. No, I know. Uh... OK. It’s just that, uh, this might be\na shock to you. - It’s longer... - How long? Please. 57 years. What? That’s the\nthing. You were out there for 57 years. You had drifted right through the core\nsystems. And it’s reallyjust blind luck... ... That a deep-salvage team found\nyou when they did. It’s one in a thousand, really. You’re damn lucky to be\nalive, kiddo. You could be floating out there forever. You OK? Uh... Oh! You\nOK? Oh! God! Nurse! Please! Someone get in here now! No! Now! Hold!\nPlease! Hold her! Kill me! No! Bad dreams again? - You wantsomething to\nhelp you sleep? - No. I’ve slept enough. Jonesy... Come here. Shh! It’s all\nright. It’s all right now. It’s over. Hi. Sorry I’m late. Been running behind all\nmorning. Is there any word about my daughter? Uh... I think we should worry\nabout the hearing now. I read your deposition... ... And it’s great. If you stick\nto that, we’ll be fine. There are gonna be a lot of heavyweights in there. Feds,\nInterstellar Commerce Commission,... ... Colonial Administration, insurance\ncompany guys. Do you have any news about my daughter? Well, we did come\nup with some information. Why don’t we sit down? I was hoping to wait\nuntil after the inquest. Uh... Amanda Ripley McClaren... Her married name, I\nguess? Age 66. And that was at the time of her death,... ... which was two years\nago. I’m real sorry. Amy... She was cremated... ... And interred at Westlake\nRepository, in Wisconsin. No children mentioned. I promised her... ... That\nI would be home for her birthday. Her eleventh birthday. Ohh... They’d like\nto go back to this point about the override destruct order. I don’t understand\nthis. We have been here for three and a half hours. How many different ways\ndo you want the same story? Look at it from our perspective, please. Please?\nYou admit to detonating the engines of, and thereby destroying, an M-Class\nstar freighter. An expensive piece of hardware. 42 million in adjusted dollars.\nThat’s minus payload, of course. The lifeboat’s flight recorder corroborates some\nelements ofyour account. In that, for reasons unknown, the Nostromo set down\non LV-426,... ... An unsurveyed planet at that time. That it resumed its course,\nand was subsequently set for self-destruct by you... ... For reasons unknown.\nNot for reasons unknown. I told you. We set down there on company orders\nto get this thing, which destroyed my crew... ... And your expensive ship. The\nanalysis team found no physical evidence ofthe creature you describe. Good!\nThat’s because I blew it out of the goddamn air lock. Like I said. Are there any\nspecies like this hostile organism on LV-426? No. It’s a rock. No indigenous\n                                          1\n"                             
 [2] "life. Did IQ’s just drop sharply while I was away? Ma’am, I already said it was\nnot indigenous. It was a derelict spacecraft. An alien ship. It was not from\nthere. Do you get it? We homed in on its beacon. And found something never\nrecorded once in over 300 surveyed worlds. A creature that ”

Criemos um vetor então que contem cada linha do filme

In [7]:
linhas=unlist(strsplit(Filme,split="\n"))

In [9]:
linhas

[[1]]
 [1] "    Bio-readouts are all in the green. Looks like she’s alive. Well, there goes our"
 [2] "salvage, guys. How are we today? Terrible. Well, better than yesterday, at least."  
 [3] "- Where am I? - You’re safe. You’re at Gateway Station. Been here a couple of"      
 [4] "days. You were pretty groggy at first, but now you’re OK. Looks like you’ve got"    
 [5] "a visitor. Jonesy! Come here! Hey, come here! How are you, you stupid cat?"         
 [6] "How are you! Where have you been? Guess you two have met, huh? I’m Burke."          
 [7] "Carter Burke. I work for the company. But... Don’t let that fool you. I’m really"   
 [8] "an OK guy. I’m glad to see you’re feeling better. They say that all the weakness"   
 [9] "and disorientation should pass soon. That’s just natural side effects of such a"    
[10] "long hypersleep. Or something like that. What do you mean? How long was I"          
[11] "out there? Has no-one discussed this with you yet? No. But, I mean, I don’t..."     
[12] "... Recognise this place. No, I know. Uh... OK. It’s just that, uh, this might be"  
[13] "a shock to you. - It’s longer... - How long? Please. 57 years. What? That’s the"    
[14] "thing. You were out there for 57 years. You had drifted right through the core"     
[15] "systems. And it’s reallyjust blind luck... ... That a deep-salvage team found"      
[16] "you when they did. It’s one in a thousand, really. You’re damn lucky to be"         
[17] "alive, kiddo. You could be floating out there forever. You OK? Uh... Oh! You"       
[18] "OK? Oh! God! Nurse! Please! Someone get in here now! No! Now! Hold!"                
[19] "Please! Hold her! Kill me! No! Bad dreams again? - You wantsomething to"            
[20] "help you sleep? - No. I’ve slept enough. Jonesy... Come here. Shh! It’s all"        
[21] "right. It’s all right now. It’s over. Hi. Sorry I’m late. Been running behind all"  
[22] "morning. Is there any word about my daughter? Uh... I think we should worry"        
[23] "about the hearing now. I read your deposition... ... And it’s great. If you stick"  
[24] "to that, we’ll be fine. There are gonna be a lot of heavyweights in there. Feds,"   
[25] "Interstellar Commerce Commission,... ... Colonial Administration, insurance"        
[26] "company guys. Do you have any news about my daughter? Well, we did come"            
[27] "up with some information. Why don’t we sit down? I was hoping to wait"              
[28] "until after the inquest. Uh... Amanda Ripley McClaren... Her married name, I"       
[29] "guess? Age 66. And that was at the time of her death,... ... which was two years"   
[30] "ago. I’m real sorry. Amy... She was cremated... ... And interred at Westlake"       
[31] "Repository, in Wisconsin. No children mentioned. I promised her... ... That"        
[32] "I would be home for her birthday. Her eleventh birthday. Ohh... They’d like"        
[33] "to go back to this point about the override destruct order. I don’t understand"     
[34] "this. We have been here for three and a half hours. How many different ways"        
[35] "do you want the same story? Look at it from our perspective, please. Please?"       
[36] "You admit to detonating the engines of, and thereby destroying, an M-Class"         
[37] "star freighter. An expensive piece of hardware. 42 million in adjusted dollars."    
[38] "That’s minus payload, of course. The lifeboat’s flight recorder corroborates some"  
[39] "elements ofyour account. In that, for reasons unknown, the Nostromo set down"       
[40] "on LV-426,... ... An unsurveyed planet at that time. That it resumed its course,"   
[41] "and was subsequently set for self-destruct by you... ... For reasons unknown."      
[42] "Not for reasons unknown. I told you. We set down there on company orders"           
[43] "to get this thing, which destroyed my crew... ... And your expensive ship. The"     
[44] "analysis team found no physical evidence ofthe creature you describe. Good!

Aqui aplicaremos o map para para cada frase isolar os termos e vermos frequencias

In [10]:
Separada=map(linhas,strsplit,split=" ")

In [11]:
Separada[[1]]

[[1]]
 [1] ""             ""             ""             ""             "Bio-readouts"
 [6] "are"          "all"          "in"           "the"          "green."      
[11] "Looks"        "like"         "she’s"        "alive."       "Well,"       
[16] "there"        "goes"         "our"         

[[2]]
 [1] "salvage,"   "guys."      "How"        "are"        "we"        
 [6] "today?"     "Terrible."  "Well,"      "better"     "than"      
[11] "yesterday," "at"         "least."    

[[3]]
 [1] "-"        "Where"    "am"       "I?"       "-"        "You’re"  
 [7] "safe."    "You’re"   "at"       "Gateway"  "Station." "Been"    
[13] "here"     "a"        "couple"   "of"      

[[4]]
 [1] "days."  "You"    "were"   "pretty" "groggy" "at"     "first," "but"   
 [9] "now"    "you’re" "OK."    "Looks"  "like"   "you’ve" "got"   

[[5]]
 [1] "a"        "visitor." "Jonesy!"  "Come"     "here!"    "Hey,"    
 [7] "come"     "here!"    "How"      "are"      "you,"     "you"     
[13] "stupid"   "cat?"    

[[6]]
 [1] "How"    "are"    "you!"   "Where"  "have"   "you"    "been?"  "Guess" 
 [9] "you"    "two"    "have"   "met,"   "huh?"   "I’m"    "Burke."

[[7]]
 [1] "Carter"   "Burke."   "I"        "work"     "for"      "the"     
 [7] "company." "But..."   "Don’t"    "let"      "that"     "fool"    
[13] "you."     "I’m"      "really"  

[[8]]
 [1] "an"       "OK"       "guy."     "I’m"      "glad"     "to"      
 [7] "see"      "you’re"   "feeling"  "better."  "They"     "say"     
[13] "that"     "all"      "the"      "weakness"

[[9]]
 [1] "and"            "disorientation" "should"         "pass"          
 [5] "soon."          "That’s"         "just"           "natural"       
 [9] "side"           "effects"        "of"             "such"          
[13] "a"             

[[10]]
 [1] "long"        "hypersleep." "Or"          "something"   "like"       
 [6] "that."       "What"        "do"          "you"         "mean?"      
[11] "How"         "long"        "was"         "I"          

[[11]]
 [1] "out"       "there?"    "Has"       "no-one"    "discussed" "this"     
 [7] "with"      "you"       "yet?"      "No."       "But,"      "I"        
[13] "mean,"     "I"         "don’t..." 

[[12]]
 [1] "..."       "Recognise" "this"      "place."    "No,"       "I"        
 [7] "know."     "Uh..."     "OK."       "It’s"      "just"      "that,"    
[13] "uh,"       "this"      "might"     "be"       

[[13]]
 [1] "a"         "shock"     "to"        "you."      "-"         "It’s"     
 [7] "longer..." "-"         "How"       "long?"     "Please."   "57"       
[13] "years."    "What?"     "That’s"    "the"      

[[14]]
 [1] "thing."  "You"     "were"    "out"     "there"   "for"     "57"     
 [8] "years."  "You"     "had"     "drifted" "right"   "through" "the"    
[15] "core"   

[[15]]
 [1] "systems."     "And"          "it’s"         "reallyjust"   "blind"       
 [6] "luck..."      "..."          "That"         "a"            "deep-salvage"
[11] "team"         "found"       

[[16]]
 [1] "you"       "when"      "they"      "did."      "It’s"      "one"      
 [7] "in"        "a"         "thousand," "really."   "You’re"    "damn"     
[13] "lucky"     "to"        "be"       

[[17]]
 [1] "alive,"   "kiddo."   "You"      "could"    "be"       "floating"
 [7] "out"      "there"    "forever." "You"      "OK?"      "Uh..."   
[13] "Oh!"      "You"     

[[18]]
 [1] "OK?"     "Oh!"     "God!"    "Nurse!"  "Please!" "Someone" "get"    
 [8] "in"      "here"    "now!"    "No!"     "Now!"    "Hold!"  

[[19]]
 [1] "Please!"       "Hold"          "her!"          "Kill"         
 [5] "me!"           "No!"           "Bad"           "dreams"       
 [9] "again?"        "-"             "You"           "wantsomething"
[13] "to"           

[[20]]
 [1] "help"      "you"       "sleep?"    "-"         "No."       "I’ve"     
 [7] "slept"     "enough."   "Jonesy..." "Come"      "here."     "Shh!"     
[13] "It’s"      "all"      

[[21]]
 [1] "right

Devemos eliminar então espaços e stopwords

In [30]:
#map(Separada,f=function(x){
 #   return(x !=" " | x !%in%% stopwords("en" ) )
#} )
Separada[[1]]!="' '"

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[46] TRUE TRUE